Para descargar mas datos necesitamos Selenium, porque Beautifulsoup no va a funcionar. Motivo la busqueda por fechas en el  sitio esta basado en Javascript.

In [3]:
import pandas as pd
import html5lib

import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.common.keys import Keys

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import time


In [5]:
page = 'https://www.camara.cl/trabajamos/sala_votaciones.aspx'
start_date = '01/01/2016'
end_date = '08/07/2017'
browser = webdriver.Firefox()

browser.get(page)
browser.find_element_by_xpath("//select[@id='ctl00_mainPlaceHolder_ddlBuscarPor']/option[@value='fecha']").click()
time.sleep(3)

inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_txtFecha1')
inputElement.clear()
time.sleep(1)
inputElement.click()
inputElement.send_keys(start_date)
inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_txtFecha2')
time.sleep(1)
inputElement.click()
inputElement.send_keys(end_date)



In [7]:
inputElement = browser.find_element_by_id('ctl00_mainPlaceHolder_btnBuscar')
inputElement.click()

In [6]:
inputElement.send_keys(end_date)